In [1]:
import sys
sys.path.append("..")

In [2]:
from keras.models import Model

Using TensorFlow backend.


In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [5]:
import bowizer
import tfidf
import pickle

In [6]:
with open('best_policy_random.pkl', 'rb') as f:
    gd = pickle.load(f)

In [7]:
gd[0][0]

'\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n          

In [8]:
blob ='\n\n\n                    ________  ________  __    __  ________        \n                   |        \\|        \\|  \\  |  \\|        \\       \n                    \\$$$$$$$$| $$$$$$$$| $$  | $$ \\$$$$$$$$       \n                      | $$   | $$__     \\$$\\/  $$   | $$          \n                      | $$   | $$  \\     >$$  $$    | $$          \n                      | $$   | $$$$$    /  $$$$\\    | $$          \n                      | $$   | $$_____ |  $$ \\$$\\   | $$          \n                      | $$   | $$     \\| $$  | $$   | $$          \n                       \\$$    \\$$$$$$$$ \\$$   \\$$    \\$$          \n              __       __   ______   _______   __        _______  \n             |  \\  _  |  \\ /      \\ |       \\ |  \\      |       \\ \n             | $$ / \\ | $$|  $$$$$$\\| $$$$$$$\\| $$      | $$$$$$$\\\n             | $$/  $\\| $$| $$  | $$| $$__| $$| $$      | $$  | $$\n             | $$  $$$\\ $$| $$  | $$| $$    $$| $$      | $$  | $$\n             | $$ $$\\$$\\$$| $$  | $$| $$$$$$$\\| $$      | $$  | $$\n             | $$$$  \\$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$\n             | $$$    \\$$$ \\$$    $$| $$  | $$| $$     \\| $$    $$\n              \\$$      \\$$  \\$$$$$$  \\$$   \\$$ \\$$$$$$$$ \\$$$$$$$ \n\n'
bloblen = len(blob)

gd[0] = [x.replace(blob, "") for x in gd[0]]

In [21]:
len_data = len(gd[0])

vocab_size = 3000
training_split = 0.7

traininglen = int(len_data * training_split)

# traininglen = 2000
training_set = gd[0][:traininglen]
test_set = gd[0][traininglen:]
test_set[0]


#tm = bowizer.TokenMaker(corpus=gd[0], vocab_size= vocab_size)



"\n-= Workshop =-\nYou've entered a workshop. You begin to take stock of what's in the room.\n\n\n\nThere is an exit to the north. Don't worry, it is unguarded. There is an exit to the west. Don't worry, it is unblocked.\n"

In [22]:
r = np.random.RandomState(42)

In [10]:
with open('norvig_tm_3000.pkl', 'rb') as f:
    tm = pickle.load(f)

In [11]:
embed_h = EmbeddingHyper(tm.vocab_size + 1, 256)
conv_h = ConvHyper(256, 6, 4)
rnn_h = RnnHyper(512, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [12]:
dernn_h = RnnHyper(512, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(256, 6, 4)
decoder_h = td.Hyper(tm.vocab_size + 1, [dernn_h, dec_h])

In [13]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [14]:
np.array(training_set).shape

ml = 128

In [17]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [18]:
testmodel = make_model(ml)

In [19]:
testmodel.load_weights('../models/wl_model64')

In [31]:
def training_round(history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train = bowizer.SlicedWordData(lines=r.choice(training_set, size=2000, replace=False), maxlen=ml,tokenmaker=tm)
    test = bowizer.SlicedWordData(lines=r.choice(test_set, size=400, replace=False), maxlen=ml,tokenmaker=tm)
    newhistory = testmodel.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=32,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [32]:
history = None

In [35]:
for i in range(20):
    print(i)
    history = training_round(history)

0
Train on 2000 samples, validate on 400 samples
Epoch 1/5
2000/2000 [==============================] - 44s - loss: 1.4378 - categorical_accuracy: 0.6869 - val_loss: 1.5296 - val_categorical_accuracy: 0.6629
Epoch 2/5
2000/2000 [==============================] - 43s - loss: 1.3764 - categorical_accuracy: 0.6920 - val_loss: 1.4511 - val_categorical_accuracy: 0.6741
Epoch 3/5
2000/2000 [==============================] - 43s - loss: 1.3099 - categorical_accuracy: 0.7008 - val_loss: 1.3845 - val_categorical_accuracy: 0.6846
Epoch 4/5
2000/2000 [==============================] - 43s - loss: 1.2866 - categorical_accuracy: 0.7049 - val_loss: 1.3736 - val_categorical_accuracy: 0.6872
Epoch 5/5
2000/2000 [==============================] - 43s - loss: 1.2631 - categorical_accuracy: 0.7068 - val_loss: 1.3212 - val_categorical_accuracy: 0.6992
1
Train on 2000 samples, validate on 400 samples
Epoch 6/10
2000/2000 [==============================] - 44s - loss: 1.2552 - categorical_accuracy: 0.7121 -

2000/2000 [==============================] - 39s - loss: 0.6901 - categorical_accuracy: 0.8103 - val_loss: 0.8512 - val_categorical_accuracy: 0.7811
Epoch 49/50
2000/2000 [==============================] - 39s - loss: 0.6661 - categorical_accuracy: 0.8147 - val_loss: 0.8568 - val_categorical_accuracy: 0.7799
Epoch 50/50
2000/2000 [==============================] - 39s - loss: 0.6548 - categorical_accuracy: 0.8169 - val_loss: 0.8767 - val_categorical_accuracy: 0.7755
10
Train on 2000 samples, validate on 400 samples
Epoch 51/55
2000/2000 [==============================] - 40s - loss: 0.7432 - categorical_accuracy: 0.7996 - val_loss: 0.8383 - val_categorical_accuracy: 0.7772
Epoch 52/55
2000/2000 [==============================] - 39s - loss: 0.7575 - categorical_accuracy: 0.7943 - val_loss: 0.8250 - val_categorical_accuracy: 0.7822
Epoch 53/55
2000/2000 [==============================] - 39s - loss: 0.6851 - categorical_accuracy: 0.8100 - val_loss: 0.8387 - val_categorical_accuracy: 0.7

2000/2000 [==============================] - 40s - loss: 0.5373 - categorical_accuracy: 0.8457 - val_loss: 0.6396 - val_categorical_accuracy: 0.8252
Epoch 97/100
2000/2000 [==============================] - 39s - loss: 0.4787 - categorical_accuracy: 0.8599 - val_loss: 0.6473 - val_categorical_accuracy: 0.8256
Epoch 98/100
2000/2000 [==============================] - 39s - loss: 0.4517 - categorical_accuracy: 0.8668 - val_loss: 0.6400 - val_categorical_accuracy: 0.8271
Epoch 99/100
2000/2000 [==============================] - 39s - loss: 0.4278 - categorical_accuracy: 0.8730 - val_loss: 0.6535 - val_categorical_accuracy: 0.8259
Epoch 100/100
2000/2000 [==============================] - 39s - loss: 0.4187 - categorical_accuracy: 0.8747 - val_loss: 0.6598 - val_categorical_accuracy: 0.8261


In [36]:
for i in range(20):
    print(i)
    history = training_round(history)

0
Train on 2000 samples, validate on 400 samples
Epoch 101/105
2000/2000 [==============================] - 41s - loss: 0.5193 - categorical_accuracy: 0.8504 - val_loss: 0.6059 - val_categorical_accuracy: 0.8377
Epoch 102/105
2000/2000 [==============================] - 40s - loss: 0.4622 - categorical_accuracy: 0.8640 - val_loss: 0.6090 - val_categorical_accuracy: 0.8384
Epoch 103/105
2000/2000 [==============================] - 40s - loss: 0.4714 - categorical_accuracy: 0.8602 - val_loss: 0.6713 - val_categorical_accuracy: 0.8227
Epoch 104/105
2000/2000 [==============================] - 40s - loss: 0.4871 - categorical_accuracy: 0.8561 - val_loss: 0.6631 - val_categorical_accuracy: 0.8232
Epoch 105/105
2000/2000 [==============================] - 40s - loss: 0.4593 - categorical_accuracy: 0.8634 - val_loss: 0.6509 - val_categorical_accuracy: 0.8299
1
Train on 2000 samples, validate on 400 samples
Epoch 106/110
2000/2000 [==============================] - 41s - loss: 0.5034 - categor

2000/2000 [==============================] - 39s - loss: 0.3016 - categorical_accuracy: 0.9083 - val_loss: 0.6395 - val_categorical_accuracy: 0.8392
Epoch 149/150
2000/2000 [==============================] - 39s - loss: 0.4114 - categorical_accuracy: 0.8787 - val_loss: 0.7303 - val_categorical_accuracy: 0.8093
Epoch 150/150
2000/2000 [==============================] - 39s - loss: 0.3738 - categorical_accuracy: 0.8870 - val_loss: 0.6335 - val_categorical_accuracy: 0.8389
10
Train on 2000 samples, validate on 400 samples
Epoch 151/155
2000/2000 [==============================] - 40s - loss: 0.4012 - categorical_accuracy: 0.8810 - val_loss: 0.5543 - val_categorical_accuracy: 0.8552
Epoch 152/155
2000/2000 [==============================] - 39s - loss: 0.3267 - categorical_accuracy: 0.9013 - val_loss: 0.5659 - val_categorical_accuracy: 0.8565
Epoch 153/155
2000/2000 [==============================] - 39s - loss: 0.2858 - categorical_accuracy: 0.9127 - val_loss: 0.5747 - val_categorical_acc

2000/2000 [==============================] - 39s - loss: 0.2259 - categorical_accuracy: 0.9310 - val_loss: 0.6147 - val_categorical_accuracy: 0.8504
19
Train on 2000 samples, validate on 400 samples
Epoch 196/200
2000/2000 [==============================] - 40s - loss: 0.2982 - categorical_accuracy: 0.9107 - val_loss: 0.4953 - val_categorical_accuracy: 0.8739
Epoch 197/200
2000/2000 [==============================] - 39s - loss: 0.2158 - categorical_accuracy: 0.9334 - val_loss: 0.5030 - val_categorical_accuracy: 0.8748
Epoch 198/200
2000/2000 [==============================] - 39s - loss: 0.1801 - categorical_accuracy: 0.9451 - val_loss: 0.5226 - val_categorical_accuracy: 0.8755
Epoch 199/200
2000/2000 [==============================] - 39s - loss: 0.1576 - categorical_accuracy: 0.9514 - val_loss: 0.5345 - val_categorical_accuracy: 0.8739
Epoch 200/200
2000/2000 [==============================] - 39s - loss: 0.1436 - categorical_accuracy: 0.9560 - val_loss: 0.5623 - val_categorical_acc

In [37]:
testmodel.save('../models/gamestate_ae128.hp5')